In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import urllib
import zipfile
import seaborn as sns

import torch
dtype = torch.float
device = torch.device("cuda:0") 
from torch.distributions import normal
import matplotlib.pyplot as plt, numpy as np, pickle
from scipy.stats import norm
from tifffile import imread
import sys
sys.path.append('/home/ashesh.ashesh/code/PPN2V')
from src.ppn2v.pn2v import *
import src.ppn2v.pn2v.gaussianMixtureNoiseModel
import src.ppn2v.pn2v.histNoiseModel
import src.ppn2v.pn2v.prediction
from src.ppn2v.pn2v.utils import *
from scripts.train_n2v import get_bestmodelname
from disentangle.core.psnr import RangeInvariantPsnr



In [ ]:
model_directory = '/home/ashesh.ashesh/training/N2V/2307/9/'
data_dir = '/group/jug/ashesh/data/ventura_gigascience/'
data_dir = data_dir[:-1] if '/' == data_dir[-1] else data_dir

dset = 'ventura_gigascience'
low_snr_filename =  'actin-60x-noise2-lowsnr.tif'
high_snr_filename = 'actin-60x-noise2-highsnr.tif'

low_snr_data = imread(os.path.join(data_dir, low_snr_filename))
high_snr_data = imread(os.path.join(data_dir, high_snr_filename))


In [ ]:
os.path.basename(data_dir)

In [ ]:
net_fpath = os.path.join(model_directory, get_bestmodelname(data_dir, low_snr_filename))
net = torch.load(net_fpath)

In [ ]:
from tqdm import tqdm 
results=[]
meanRes=[]
resultImgs=[]
inputImgs=[]
dataTest = low_snr_data

for index in tqdm(range(dataTest.shape[0])):

    im=dataTest[index]
    # We are using tiling to fit the image into memory
    # If you get an error try a smaller patch size (ps)
    means = src.ppn2v.pn2v.prediction.tiledPredict(im, net, ps=256, overlap=48,
                                            device=device, noiseModel=None)
    resultImgs.append(means)
    inputImgs.append(im)
    
prediction = np.concatenate([x[None] for x in resultImgs],axis=0).astype(np.int32)

In [ ]:
_,ax = plt.subplots(figsize=(12,4),ncols=3)
img_idx = np.random.randint(0,high=len(inputImgs))
hs = 0
ws = 0
hN = 1500
wN = 1500
ax[0].imshow(inputImgs[img_idx][hs:hs+hN, ws:ws+wN])
ax[1].imshow(resultImgs[img_idx][hs:hs+hN, ws:ws+wN])
ax[2].imshow(high_snr_data[img_idx][hs:hs+hN, ws:ws+wN])

In [ ]:
psnrs = RangeInvariantPsnr(high_snr_data.astype(np.int32), prediction)
print(f'{torch.mean(psnrs).item():.1f}','+-', f'{torch.std(psnrs).item():.1f}')